# 🚀 LPU Bot - Universal Runner (V8 - Deployment Ready)
**Works on:** Google Colab Cloud (Linux) AND VS Code (Windows/Mac).

In [ ]:
# 1. Setup Environment
import platform, os, sys, subprocess
IS_WINDOWS = (platform.system() == 'Windows')
print(f"🖥️ OS Detected: {platform.system()}")

if IS_WINDOWS:
    print("ℹ️ Windows Mode: Skipping Linux installation.")
else:
    print("ℹ️ Linux Mode: Installing Ollama...")
    !curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# 2. Start Ollama
import time
if not IS_WINDOWS:
    print("⏳ Starting Ollama...")
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)
    print("✅ Ollama Running")
else:
    print("✅ Assumed running on Windows.")

In [ ]:
# 3. Pull Model
print("⬇️ Pulling gemma2:2b...")
!ollama pull gemma2:2b

In [ ]:
# 4. Unzip Project
import os
import zipfile

if not os.path.exists('project.zip'):
    print("⚠️ Project.zip NOT FOUND!")
    print("👉 Uplodad 'project.zip' via the interface!")
else:
    print("📦 Unzipping project...")
    with zipfile.ZipFile("project.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    print("✅ Unzip Done!")

if os.path.exists("src/llm_agent.py"):
    print("✅ Code verified.")
else:
    print("❌ ERROR: src/llm_agent.py missing. Zip might be empty/wrong.")

In [ ]:
# 5. Install Dependencies
!pip install -r requirements.txt

In [ ]:
# 6. Ingest Data
!python src/ingest.py

In [ ]:
# 7. Run Agent (Console Mode)
import sys, os, asyncio, platform, subprocess
import requests
from contextlib import AsyncExitStack
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# --- HEADER: PRE-FLIGHT CHECKS ---
print("🔍 Running Pre-Flight Checks...")

# 1. Check Source Code
if not os.path.exists('src/llm_agent.py'):
    raise FileNotFoundError("❌ CODE MISSING: src/llm_agent.py not found. Please run Step 4 (Unzip) again.")

# 2. Check Ollama Server
try:
    response = requests.get("http://127.0.0.1:11434", timeout=2)
    if response.status_code == 200:
        print("✅ Ollama Server is UP.")
    else:
        print(f"⚠️ Ollama responded with {response.status_code}")
except:
    print("❌ OLLAMA DOWN: Connection refused.")
    print("👉 FIX: Please Run Step 2 (Start Ollama) again. It stops when Colab restarts.")
    raise ConnectionError("Ollama not running.")

# 3. Check Model
try:
    m_resp = requests.get("http://127.0.0.1:11434/api/tags")
    models = [m['name'] for m in m_resp.json()['models']]
    if "gemma2:2b" not in models:
        print(f"⚠️ WARNING: gemma2:2b not found in {models}")
        print("👉 FIX: Run Step 3 (Pull Model) again.")
    else:
        print("✅ Model 'gemma2:2b' is ready.")
except:
    pass

# --- END CHECKS ---

if platform.system() == 'Windows':
    try: asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    except: pass

if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

class SafeUniMcpClient:
    def __init__(self):
        self.session = None
        self.exit_stack = AsyncExitStack()
    async def __aenter__(self):
        params = StdioServerParameters(
            command=sys.executable,
            args=[os.path.abspath("src/mcp_server.py")],
            env=os.environ.copy()
        )
        try:
            read, write = await self.exit_stack.enter_async_context(stdio_client(params, errlog=subprocess.DEVNULL))
            self.session = await self.exit_stack.enter_async_context(ClientSession(read, write))
            await self.session.initialize()
            return self
        except Exception as e:
            await self.exit_stack.aclose()
            raise e
    async def __aexit__(self, exc_type, exc_val, exc_tb): await self.exit_stack.aclose()
    async def get_tools(self): return (await self.session.list_tools()).tools
    async def call_tool(self, name, args): return await self.session.call_tool(name, arguments=args)

import llm_agent
import importlib
importlib.reload(llm_agent)
llm_agent.UniMcpClient = SafeUniMcpClient
agent = llm_agent.UniAgent()
print("🤖 Bot active (Console Mode). Type message below.")
await agent.chat_loop()

In [ ]:
# 8. Launch UI (Gradio)
# This will give you a PUBLIC URL (e.g. https://1234.gradio.live)
!python web_app.py